In [39]:
# Numerical analysis module
import numpy as np
# Plot module
import matplotlib.pyplot as plt
import pandas as pd
# read netCDF files
import xarray as xr
import datetime
import openpyxl
import pyomo.environ as pyo
from scipy.optimize import minimize
import pygame

ModuleNotFoundError: No module named 'pygame'

To Do list Données: 
-Mettre les paramètres dans un dictionnaire 
-Créer la fonction calcul de cout 


# Paramètres de l'optimisation

#### Capacité de production

In [40]:
#Generator 
solar_gen = 0  #MW
wind_gen = 30 #Mw
diesel_gen = 24 #Mw

#Storage 
Energy = 0 ##MWh
charging_power = Energy/3  #MW

#Aleady installed Diesel 
diesel_param =16
stock0=0
capacity_data = {'solar_gen':solar_gen,'wind_gen':wind_gen,'diesel_gen':diesel_gen,'Energy':Energy,'diesel_param':diesel_param,'stock0':stock0}


#### Paramètres de cout

In [61]:
#taux d'actualisation

t_act = 0.05 #taux d'actualisation des coûts de 4%, celui de RTE

#Variable Cost
unserved_energy_cost= 3000#€/Mwh 
diesel_variable_cost = 177 #€/Mwh

#Lithium Ion battery characterics
specific_energy = 105 #Mwh/kg
specific_power = 315 #Mw/kg
ratio_ener_to_pow = specific_energy/specific_power

# Initial Cost of new capacity 
solar_cost =  747000  #€/Mw
wind_cost =  1300000     #€/Mw
#diesel_cost =  1050505     #€/Mw à actualiser
diesel_cost = 1050505*((1.05)**(19))
#storage_energy_cost = 1167144   #€/MWh à actualiser
storage_energy_cost = 1167144
storage_power_cost = 0 #storage_energy_cost*ratio_ener_to_pow  #€/Mw

# O&M : Operation & Maintenance 
solar_om_cost =  11000    #€/MW.an
wind_om_cost =  40000    #€/MW.an
#diesel_om_cost =  101000    #€/MW.an à actualisaer
diesel_om_cost = 101000*((1.05)**(19))
CO2_cost=100# 44.6  #€/tCO2
#NVP  taux d'actualisation
nvp = 12.46

cost_data = {'unserved_energy_cost':unserved_energy_cost,'diesel_variable_cost':diesel_variable_cost,'solar_cost':solar_cost,'wind_cost':wind_cost,'diesel_cost':diesel_cost,'storage_power_cost':storage_power_cost,'storage_energy_cost':storage_energy_cost,'solar_om_cost':solar_om_cost,'wind_om_cost':wind_om_cost,'diesel_om_cost':diesel_om_cost,'nvp':nvp,'CO2_cost':CO2_cost}

In [62]:
# Emission de CO2
diesel_co2 = 0.769 #tco2/Mwh

# Extraction des données

In [40]:
def finisher() :
    pygame.mixer.init()
    pygame.mixer.music.load("mario.mp3")
    pygame.mixer.music.play()
finisher()

In [63]:
data= pd.read_excel('./EA314_project_isolated-system.xlsx',header=2,sheet_name='Case 2',usecols='E:P')[1:-1]
climate_data = pd.read_excel('./EA314_project_isolated-system.xlsx',header=2,sheet_name='Case 2',usecols='T:U')
climate = climate_data[1:-1]


In [64]:
wind_fc= xr.DataArray(
data=climate['WIND_FC'],
dims = ['time'],
coords = dict(time=pd.to_datetime(data['Time (UTC)'])),attrs =dict(description = 'Wind context data', units ='t')  
    )
solar_fc= xr.DataArray(
data=climate['SOLAR_FC'],
dims = ['time'],
coords = dict(time=pd.to_datetime(data['Time (UTC)'])),attrs =dict(description = 'Radiation context data', units ='t')  
    )

load= xr.DataArray(
data=data['Load'],
dims = ['time'],
coords = dict(time=pd.to_datetime(data['Time (UTC)'])),attrs =dict(description = 'Demande en MWH')  
    )
context = xr.merge([load,wind_fc,solar_fc])

## Mots clefs pour bool 
Débordement : Trop de vent et de soleil 


Les 5 jours de rab : Opacité Météorologique



In [65]:
def compute_data(context,capacity_data):
    ## Charging DATA
    solar_gen = capacity_data['solar_gen']
    wind_gen = capacity_data['wind_gen']
    diesel_gen = capacity_data['diesel_gen']
    
    Energy=capacity_data['Energy']
    charging_power = Energy/3
    solar_prod = context.Load.copy()
    solar_prod = context.SOLAR_FC*solar_gen
    solar_prod.name = 'solar_production'
    wind_prod = context.Load.copy()
    wind_prod = context.WIND_FC*wind_gen
    wind_prod.name = "wind_production"
    
    ##Creating structure for our data
    net_load= context.Load.copy()
    net_load = context.Load -solar_prod - wind_prod
    net_load.name = 'net_load'
    missing_capacity = net_load.copy() - diesel_gen
    missing_capacity.name = 'missing_capacity'
    missing_capacity.values = (abs(missing_capacity.values)+missing_capacity.values)/2
    diesel = context.Load.copy()
    diesel.name='diesel'
    unserved_energy = context.Load.copy()
    unserved_energy.name = 'unserved_energy'
    unused_energy = context.Load.copy()
    unused_energy.name = 'unused_energy'
    charging = context.Load.copy()
    charging.name = 'charging'
    stock = context.Load.copy()
    stock.name = 'stock'
    test = context.Load.copy()
    releasing = context.Load.copy()
    releasing.name='releasing'
    
    ## Computing Storage data
    for i in range(0,len(charging.values)):
        if i == 0:
            stocki =stock0
        else :
            stocki =stock.values[i-1]

        remain_energy = net_load.values[i]<0 #BOOL 
        five_day_missing_cap =np.sum(missing_capacity.isel(time=slice(i+1,i+5*24+1)).values)
        missing_storage = five_day_missing_cap-stocki>0.0001 #BOOL   la condition ne fonctionnait pas avec une inégalité python
        five_day_missing_capacity = five_day_missing_cap>0
        one_hour_missing_capacity = missing_capacity.values[i]>0
        diesel_charging = np.min([five_day_missing_cap , diesel_gen-net_load.values[i] ,Energy-stocki])
        charging.values[i]=np.min([charging_power,remain_energy*(np.min([-net_load.values[i],Energy-stocki]))+
                                (1-remain_energy)*(missing_storage*(np.max([diesel_charging,0])))])
        releasing.values[i]= np.min([charging_power,one_hour_missing_capacity*np.min([missing_capacity.values[i],stocki]) 
                                +(1-one_hour_missing_capacity)*(1-five_day_missing_capacity)*
                                np.min([0.5*(np.abs(net_load.values[i])+net_load.values[i]),stocki])])
        stock.values[i]= stocki + charging.values[i]-releasing.values[i]
        diesel.values[i] = np.min([diesel_gen,np.max([net_load.values[i]+charging.values[i]-releasing.values[i],0])])

    for i in range(len(diesel)):
        diesel.values[i] = np.min([diesel_gen,np.max([net_load.values[i]+charging.values[i]-releasing.values[i],0])])

    unserved_energy.values = 0.5*(np.abs(net_load.values+charging.values-releasing.values-diesel.values)+(net_load.values+charging.values-releasing.values-diesel.values))
    unused_energy.values = 0.5*(np.abs(-net_load.values-charging.values)-net_load.values-charging.values)
    energy_scenario = context.Load.copy()
    energy_scenario = xr.merge([context.Load,solar_prod,wind_prod,net_load,missing_capacity,charging,releasing,stock,diesel,unserved_energy,unused_energy])
    energy_scenario.attrs =dict(description ="Caractéristiques du parc énergétique sur l'intervalle de temps donné",units='Mwh')
    return(energy_scenario)
energy_scenario = compute_data(context,capacity_data)
energy_scenario

<xarray.Dataset>
Dimensions:           (time: 8735)
Coordinates:
  * time              (time) datetime64[ns] 2000-06-30T22:00:00 ... 2001-06-2...
Data variables:
    Load              (time) float64 11.0 10.0 10.0 9.0 ... 13.0 12.0 11.0 12.0
    solar_production  (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    wind_production   (time) float64 6.9 6.9 5.7 3.6 3.0 ... 9.3 9.9 7.8 7.2
    net_load          (time) float64 4.1 3.1 4.3 5.4 6.0 ... 2.5 3.7 2.1 3.2 4.8
    missing_capacity  (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    charging          (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    releasing         (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    stock             (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    diesel            (time) float64 4.1 3.1 4.3 5.4 6.0 ... 2.5 3.7 2.1 3.2 4.8
    unserved_energy   (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    unused_energy     (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
Attributes:
    description:  Caractéristiques du parc énergétique sur l'intervalle de te...
    units:        Mwh

In [66]:
def compute_data_opt(context,solar_gen,wind_gen,diesel_gen,Energy):
    ## Charging DATA
#     solar_gen = capacity_data['solar_gen']
#     wind_gen = capacity_data['wind_gen']
#     diesel_gen = capacity_data['diesel_gen']
#     charging_power = capacity_data['charging_power']
#     Energy=capacity_data['Energy']
    solar_prod = context.Load.copy()
    solar_prod = context.SOLAR_FC*solar_gen
    solar_prod.name = 'solar_production'
    wind_prod = context.Load.copy()
    wind_prod = context.WIND_FC*wind_gen
    wind_prod.name = "wind_production"
    charging_power= Energy/3
    ##Creating structure for our data
    net_load= context.Load.copy()
    net_load = context.Load -solar_prod - wind_prod
    net_load.name = 'net_load'
    missing_capacity = net_load.copy() - diesel_gen
    missing_capacity.name = 'missing_capacity'
    missing_capacity.values = (abs(missing_capacity.values)+missing_capacity.values)/2
    diesel = context.Load.copy()
    diesel.name='diesel'
    unserved_energy = context.Load.copy()
    unserved_energy.name = 'unserved_energy'
    unused_energy = context.Load.copy()
    unused_energy.name = 'unused_energy'
    charging = context.Load.copy()
    charging.name = 'charging'
    stock = context.Load.copy()
    stock.name = 'stock'
    test = context.Load.copy()
    releasing = context.Load.copy()
    releasing.name='releasing'
    
    ## Computing Storage data
    for i in range(0,len(charging.values)):
        if i == 0:
            stocki =stock0
        else :
            stocki =stock.values[i-1]

        remain_energy = net_load.values[i]<0 #BOOL 
        five_day_missing_cap =np.sum(missing_capacity.isel(time=slice(i+1,i+5*24+1)).values)
        missing_storage = five_day_missing_cap-stocki>0.0001 #BOOL   la condition ne fonctionnait pas avec une inégalité python
        five_day_missing_capacity = five_day_missing_cap>0
        one_hour_missing_capacity = missing_capacity.values[i]>0
        diesel_charging = np.min([five_day_missing_cap , diesel_gen-net_load.values[i] ,Energy-stocki])
        charging.values[i]=np.min([charging_power,remain_energy*(np.min([-net_load.values[i],Energy-stocki]))+
                                (1-remain_energy)*(missing_storage*(np.max([diesel_charging,0])))])
        releasing.values[i]= np.min([charging_power,one_hour_missing_capacity*np.min([missing_capacity.values[i],stocki]) 
                                +(1-one_hour_missing_capacity)*(1-five_day_missing_capacity)*
                                np.min([0.5*(np.abs(net_load.values[i])+net_load.values[i]),stocki])])
        stock.values[i]= stocki + charging.values[i]-releasing.values[i]
        diesel.values[i] = np.min([diesel_gen,np.max([net_load.values[i]+charging.values[i]-releasing.values[i],0])])

    for i in range(len(diesel)):
        diesel.values[i] = np.min([diesel_gen,np.max([net_load.values[i]+charging.values[i]-releasing.values[i],0])])

    unserved_energy.values = 0.5*(np.abs(net_load.values+charging.values-releasing.values-diesel.values)+(net_load.values+charging.values-releasing.values-diesel.values))
    unused_energy.values = 0.5*(np.abs(-net_load.values-charging.values)-net_load.values-charging.values)
    energy_scenario = context.Load.copy()
    energy_scenario = xr.merge([context.Load,solar_prod,wind_prod,net_load,missing_capacity,charging,releasing,stock,diesel,unserved_energy,unused_energy])
    energy_scenario.attrs =dict(description ="Caractéristiques du parc énergétique sur l'intervalle de temps donné",units='Mwh')
    return(energy_scenario)
energy_scenario_opt =compute_data_opt(context,solar_gen,wind_gen,diesel_gen,Energy)
energy_scenario_opt

<xarray.Dataset>
Dimensions:           (time: 8735)
Coordinates:
  * time              (time) datetime64[ns] 2000-06-30T22:00:00 ... 2001-06-2...
Data variables:
    Load              (time) float64 11.0 10.0 10.0 9.0 ... 13.0 12.0 11.0 12.0
    solar_production  (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    wind_production   (time) float64 6.9 6.9 5.7 3.6 3.0 ... 9.3 9.9 7.8 7.2
    net_load          (time) float64 4.1 3.1 4.3 5.4 6.0 ... 2.5 3.7 2.1 3.2 4.8
    missing_capacity  (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    charging          (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    releasing         (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    stock             (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    diesel            (time) float64 4.1 3.1 4.3 5.4 6.0 ... 2.5 3.7 2.1 3.2 4.8
    unserved_energy   (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    unused_energy     (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
Attributes:
    description:  Caractéristiques du parc énergétique sur l'intervalle de te...
    units:        Mwh

# Calculs macro

#### Couts

In [67]:
#Unité chelou par rapport à l'excel (x10 pour le charging et l'énergy)
def compute_cost(context,capacity_data,cost_data) :
    #Charging Data 
    unserved_energy_cost=cost_data['unserved_energy_cost']
    diesel_variable_cost=cost_data['diesel_variable_cost']
    solar_cost=cost_data['solar_cost']
    wind_cost=cost_data['wind_cost']
    diesel_cost=cost_data['diesel_cost']
    storage_power_cost=cost_data['storage_power_cost']
    storage_energy_cost=cost_data['storage_energy_cost']
    solar_om_cost=cost_data['solar_om_cost']
    wind_om_cost=cost_data['wind_om_cost']
    diesel_om_cost=cost_data['diesel_om_cost']
    nvp = cost_data['nvp']
    energy_scenario = compute_data(context,capacity_data)
    total_cost_prod = solar_cost*solar_gen + wind_cost*wind_gen + diesel_cost*np.max([diesel_gen-diesel_param,0])+storage_power_cost*charging_power +storage_energy_cost*Energy 
    total_cost_om = solar_om_cost*solar_gen + wind_om_cost*wind_gen +diesel_om_cost*diesel_gen
    total_fuel_cost = np.sum(energy_scenario.diesel.values)*diesel_variable_cost
    total_lole_cost = unserved_energy_cost*np.sum(energy_scenario.unserved_energy.values)
    emissions_co2 =(np.sum(energy_scenario.diesel.values))*diesel_co2
    CO2_cost = emissions_co2*cost_data['CO2_cost']

    total_cost_2021 =total_cost_prod+ (total_cost_om + total_fuel_cost + total_lole_cost)*nvp 
    lcoe = total_cost_2021/(nvp*context.Load.values.sum())
    final_cost_dic = {'total_cost_2021':total_cost_2021*1e-6,'total_cost_prod':total_cost_prod*1e-6,'total_cost_om_2021':total_cost_om*1e-6,'total_fuel_cost_2021':total_fuel_cost*1e-6,'total_lole_cost_2021':total_lole_cost*1e-6,'LCOE':lcoe,'CO2 Cost':CO2_cost*1e-6,'emissions co2':emissions_co2}
#     final_cost =pd.DataFrame([0,1,2,3],final_cost_dic)
    return(final_cost_dic)

compute_cost(context,capacity_data,cost_data)

{'total_cost_2021': 322.29458501888297,
 'total_cost_prod': 60.23659051994469,
 'total_cost_om_2021': 7.325327273590552,
 'total_fuel_cost_2021': 13.706614499999997,
 'total_lole_cost_2021': 0.0,
 'LCOE': 198.38127333697162,
 'CO2 Cost': 5.955020649999999,
 'emissions co2': 59550.20649999999}

In [69]:

#Unité chelou par rapport à l'excel (x10 pour le charging et l'énergy)
def compute_cost_opt(prod_opt,cost_data,context,diesel_co2) :
    #Charging Data 
    unserved_energy_cost=cost_data['unserved_energy_cost']
    diesel_variable_cost=cost_data['diesel_variable_cost']
    solar_cost=cost_data['solar_cost']
    wind_cost=cost_data['wind_cost']
    diesel_cost=cost_data['diesel_cost']
    storage_power_cost=cost_data['storage_power_cost']
    storage_energy_cost=cost_data['storage_energy_cost']
    solar_om_cost=cost_data['solar_om_cost']
    wind_om_cost=cost_data['wind_om_cost']
    diesel_om_cost=cost_data['diesel_om_cost']
    nvp = cost_data['nvp']
    [solar_gen,wind_gen,diesel_gen,Energy] = prod_opt
    energy_scenario = compute_data_opt(context,solar_gen,wind_gen,diesel_gen,Energy)
    total_cost_prod = solar_cost*solar_gen + wind_cost*wind_gen + diesel_cost*np.max([diesel_gen-diesel_param,0]) +storage_energy_cost*Energy 
    total_cost_om = solar_om_cost*solar_gen + wind_om_cost*wind_gen +diesel_om_cost*diesel_gen
    total_fuel_cost = np.sum(energy_scenario.diesel.values)*diesel_variable_cost
    total_lole_cost = unserved_energy_cost*np.sum(energy_scenario.unserved_energy.values)
    #Ajout d'un cout carbone
    CO2_cost = np.sum(energy_scenario.diesel.values)*diesel_co2*cost_data['CO2_cost']
    ###########
    total_cost_2021 =total_cost_prod+  (total_cost_om + total_fuel_cost + total_lole_cost+CO2_cost)*nvp 
    final_cost_dic = {'total_cost_2021':total_cost_2021*1e-6,'total_cost_prod':total_cost_prod*1e-6,'total_cost_om_2021':total_cost_om*nvp*1e-6,'total_fuel_cost_2021':total_fuel_cost*nvp*1e-6,'total_lole_cost_2021':total_lole_cost*nvp*1e-6}
#     final_cost =pd.DataFrame([0,1,2,3],final_cost_dic)
    return(total_cost_2021)
prod_opt =  [solar_gen,wind_gen,diesel_gen,Energy]

compute_cost_opt(prod_opt,cost_data,context,diesel_co2)

396494142.3178829

#### Calcul des émissions de CO2

In [70]:
emission_diesel = np.sum(energy_scenario_opt.diesel.values)*diesel_co2
emission_diesel

59550.20649999999

# Optimisation

##### Credit to Laura Ladislas

In [50]:
variables_test = [42,50,24,68]
bornes = [(0,100), (0,100), (0,100), (0,100)]
resultat_optimisation = minimize(compute_cost_opt,[20,20,20,10], method='Nelder-Mead', args=(cost_data,context,diesel_co2), bounds=bornes)
prod_opt = resultat_optimisation.x
capacity_data = {'solar_gen':prod_opt[0],'wind_gen':prod_opt[1],'diesel_gen':prod_opt[2],'Energy':prod_opt[3],'diesel_param':diesel_param,'stock0':stock0}
compute_cost_opt(prod_opt,cost_data,context,diesel_co2)
[solar_gen,wind_gen,diesel_gen,Energy]=prod_opt
energy_scenario_opt =compute_data_opt(context,solar_gen,wind_gen,diesel_gen,Energy)


cost_opt = compute_cost(context,capacity_data,cost_data) 
energy_scenario_opt
finisher() ## Met une petite musique sympas pour annoncer la fin du run 

IndexError: index 4 is out of bounds for axis 0 with size 4

In [51]:
resultat_optimisation.x

array([27.090303  , 76.92307888, 18.96153841, 12.8076924 ])

In [54]:
capacity_data = {'solar_gen':prod_opt[0],'wind_gen':prod_opt[1],'diesel_gen':prod_opt[2],'Energy':prod_opt[3],'diesel_param':diesel_param,'stock0':stock0}
compute_cost_opt(prod_opt,cost_data,context,diesel_co2)
[solar_gen,wind_gen,diesel_gen,Energy]=prod_opt
energy_scenario_opt =compute_data_opt(context,solar_gen,wind_gen,diesel_gen,Energy)


cost_opt = compute_cost(context,capacity_data,cost_data) 
energy_scenario_opt

<xarray.Dataset>
Dimensions:           (time: 8735)
Coordinates:
  * time              (time) datetime64[ns] 2000-06-30T22:00:00 ... 2001-06-2...
Data variables:
    Load              (time) float64 11.0 10.0 10.0 9.0 ... 13.0 12.0 11.0 12.0
    solar_production  (time) float64 0.0 0.0 0.0 0.0 ... 4.605 1.896 0.2709
    wind_production   (time) float64 17.69 17.69 14.62 ... 25.38 20.0 18.46
    net_load          (time) float64 -6.692 -7.692 -4.615 ... -10.9 -6.732
    missing_capacity  (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    charging          (time) float64 4.269 4.269 4.269 0.0 ... 0.0 0.0 0.0 0.0
    releasing         (time) float64 0.0 0.0 0.0 0.0 1.308 ... 0.0 0.0 0.0 0.0
    stock             (time) float64 4.269 8.538 12.81 ... 12.81 12.81 12.81
    diesel            (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    unserved_energy   (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    unused_energy     (time) float64 2.423 3.423 0.3462 ... 17.99 10.9 6.732
Attributes:
    description:  Caractéristiques du parc énergétique sur l'intervalle de te...
    units:        Mwh

In [55]:
cost_opt

{'total_cost_2021': 270.867941134039,
 'total_cost_prod': 146.96488809706588,
 'total_cost_om_2021': 5.533844594973719,
 'total_fuel_cost_2021': 4.381424670254264,
 'total_lole_cost_2021': 0.028795986535504133,
 'LCOE': 166.726744928669,
 'CO2 Cost': 7.614272477797806,
 'emissions co2': 19035.681194494515}

In [59]:
file = open("Sorties/test.txt", "w+")

# Saving the array in a text file
content = str(cost_opt)
file.write('[Solar,Wind,Diesel,Energy] \n \n : ')
file.write(str(prod_opt))
file.write('\n \n Couts du résultat optimisé \n \n : ')
file.write(content)

file.close()

energy_scenario_opt.to_netcdf('Sorties/test.nc', 'w')
# test = xr.open_dataset('1')
# test

# Runs Pour les 3 scenarios 
3 scenarios : 
_ Cout du co2 à 45€ la tonne 
_Cout du CO2 à 100€ la tonne 
_ 0 diesel avec les couts de 2040- 2050 


#### Péliminaires 


In [50]:
## Vérifier que les données sont bien chargées (voir les premières cellules) ##
print(cost_data)
print(context)
print(diesel_co2)
## A modifier avec vos données
bornes = [(0,150), (0,150), (0,150), (0,150)]


{'unserved_energy_cost': 3000, 'diesel_variable_cost': 177, 'solar_cost': 747000, 'wind_cost': 1300000, 'diesel_cost': 2787302.5057427413, 'storage_power_cost': 0, 'storage_energy_cost': 3096780.4967730814, 'solar_om_cost': 11000, 'wind_om_cost': 40000, 'diesel_om_cost': 267983.0682195866, 'nvp': 12.46, 'CO2_cost': 100}
<xarray.Dataset>
Dimensions:   (time: 8735)
Coordinates:
  * time      (time) datetime64[ns] 2000-06-30T22:00:00 ... 2001-06-29T20:00:00
Data variables:
    Load      (time) float64 11.0 10.0 10.0 9.0 9.0 ... 13.0 13.0 12.0 11.0 12.0
    WIND_FC   (time) float64 0.23 0.23 0.19 0.12 0.1 ... 0.31 0.33 0.26 0.24
    SOLAR_FC  (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.38 0.26 0.17 0.07 0.01
Attributes:
    description:  Demande en MWH
0.769


### Run 1

In [105]:
cost_data['CO2_cost'] = 45
resultat_optimisation = minimize(compute_cost_opt,[20,20,20,10], method='Nelder-Mead', args=(cost_data,context,diesel_co2), bounds=bornes)
prod_opt = resultat_optimisation.x
capacity_data = {'solar_gen':prod_opt[0],'wind_gen':prod_opt[1],'diesel_gen':prod_opt[2],'Energy':prod_opt[3],'diesel_param':diesel_param,'stock0':stock0}
[solar_gen,wind_gen,diesel_gen,Energy]=prod_opt
# Calcul des couts du scenario optimisé
cost_opt = compute_cost(context,capacity_data,cost_data) 
# Calcul du scenario énergétique 
energy_scenario_opt =compute_data_opt(context,solar_gen,wind_gen,diesel_gen,Energy)

## Writing on xls
with pd.ExcelWriter('run1.xlsx') as writer :
    capacity_data_frame = (pd.DataFrame(data=capacity_data, index=[0])).T
    cost_opt_frame = (pd.DataFrame(data=cost_opt, index=[0])).T
    capacity_data_frame.to_excel(writer ,sheet_name="Run 1 Capacity Data")
    cost_opt_frame.to_excel(writer ,sheet_name="Run 1 Optimised Cost")

{'unserved_energy_cost': 3000,
 'diesel_variable_cost': 177,
 'solar_cost': 747000,
 'wind_cost': 1300000,
 'diesel_cost': 1050505,
 'storage_power_cost': 0,
 'storage_energy_cost': 1167144,
 'solar_om_cost': 11000,
 'wind_om_cost': 40000,
 'diesel_om_cost': 101000,
 'nvp': 12.46,
 'CO2_cost': 45}

### Run 2

In [ ]:
cost_data['CO2_cost'] = 100
resultat_optimisation = minimize(compute_cost_opt,[20,20,20,10], method='Nelder-Mead', args=(cost_data,context,diesel_co2), bounds=bornes)
prod_opt = resultat_optimisation.x
capacity_data = {'solar_gen':prod_opt[0],'wind_gen':prod_opt[1],'diesel_gen':prod_opt[2],'Energy':prod_opt[3],'diesel_param':diesel_param,'stock0':stock0}
[solar_gen,wind_gen,diesel_gen,Energy]=prod_opt
# Calcul des couts du scenario optimisé
cost_opt = compute_cost(context,capacity_data,cost_data) 
# Calcul du scenario énergétique 
energy_scenario_opt =compute_data_opt(context,solar_gen,wind_gen,diesel_gen,Energy)

## Writing on xls
with pd.ExcelWriter('run2.xlsx') as writer :
    capacity_data_frame = (pd.DataFrame(data=capacity_data, index=[0])).T
    cost_opt_frame = (pd.DataFrame(data=cost_opt, index=[0])).T
    capacity_data_frame.to_excel(writer ,sheet_name="Run 2 Capacity Data")
    cost_opt_frame.to_excel(writer ,sheet_name="Run 2 Optimised Cost")

### Run 3

In [71]:
# Couts à rentrer 
cost_data['CO2_cost'] = 100 
cost_data['wind_cost'] = 1050000*((1.05)/1.04)**(19)
cost_data['solar_cost'] = 517000*((1.05)/1.04)**(19)
cost_data['wind_om_cost'] = 30000*((1.05)/1.04)**(19)
cost_data['solar_om_cost'] = 9000*((1.05)/1.04)**(19)
cost_data['storage_energy_cost'] = 1167144*((1.05)**(19))
bornes[2]= (0,1)


resultat_optimisation = minimize(compute_cost_opt,[20,20,0,10], method='Nelder-Mead', args=(cost_data,context,diesel_co2), bounds=bornes)
prod_opt = resultat_optimisation.x
capacity_data = {'solar_gen':prod_opt[0],'wind_gen':prod_opt[1],'diesel_gen':prod_opt[2],'Energy':prod_opt[3],'diesel_param':diesel_param,'stock0':stock0}
[solar_gen,wind_gen,diesel_gen,Energy]=prod_opt
# Calcul des couts du scenario optimisé
cost_opt = compute_cost(context,capacity_data,cost_data) 
# Calcul du scenario énergétique 
energy_scenario_opt =compute_data_opt(context,solar_gen,wind_gen,diesel_gen,Energy)

## Writing on xls
with pd.ExcelWriter('run3.xlsx') as writer :
    capacity_data_frame = (pd.DataFrame(data=capacity_data, index=[0])).T
    cost_opt_frame = (pd.DataFrame(data=cost_opt, index=[0])).T
    capacity_data_frame.to_excel(writer ,sheet_name="Run 3 Capacity Data")
    cost_opt_frame.to_excel(writer ,sheet_name="Run 3 Optimised Cost")

### Sensibilité CO2

In [99]:
L_cost_opt =[]
L_prod_opt = []

In [100]:
diesel0=24
dieselbounds = [(0,24),(0,0.75*diesel0),(0,0.5*diesel0),(0,0.25*diesel0),(0,0)]
bornes = [(0,150), (0,150), (0,150), (0,150)]
cost_data['CO2_cost'] = 0
for borne_diesel in dieselbounds : 
    bornes[2]=borne_diesel
    resultat_optimisation = minimize(compute_cost_opt,[20,20,20,10], method='Nelder-Mead', args=(cost_data,context,diesel_co2), bounds=bornes)
    prod_opt = resultat_optimisation.x
    capacity_data = {'solar_gen':prod_opt[0],'wind_gen':prod_opt[1],'diesel_gen':prod_opt[2],'Energy':prod_opt[3],'diesel_param':diesel_param,'stock0':stock0}
    [solar_gen,wind_gen,diesel_gen,Energy]=prod_opt
    # Calcul des couts du scenario optimisé
    cost_opt = compute_cost(context,capacity_data,cost_data) 
    # Calcul du scenario énergétique 
    energy_scenario_opt =compute_data_opt(context,solar_gen,wind_gen,diesel_gen,Energy)
    
    L_cost_opt.append(cost_opt)
    L_prod_opt.append(prod_opt)
    file = open("Sorties/{}diesel_max.txt".format(borne_diesel[1]), "w+")

    # Saving the array in a text file
    file.write(str(cost_data))
    content = str(cost_opt)
    file.write('\n \n [Solar,Wind,Diesel,Energy] \n \n : ')
    file.write(str(prod_opt))
    file.write('\n \n Couts du résultat optimisé \n \n : ')
    file.write(content)
    
    file.close()

    energy_scenario_opt.to_netcdf("Sorties/{}diesel_max.nc".format(borne_diesel[1]), 'w')
    
    print(co2_cost)
finisher()

1000


C:\Users\augus\AppData\Local\Temp\ipykernel_22568\3013852297.py:7: OptimizeWarning: Initial guess is not within the specified bounds
  resultat_optimisation = minimize(compute_cost_opt,[20,20,20,10], method='Nelder-Mead', args=(cost_data,context,diesel_co2), bounds=bornes)


1000


C:\Users\augus\AppData\Local\Temp\ipykernel_22568\3013852297.py:7: OptimizeWarning: Initial guess is not within the specified bounds
  resultat_optimisation = minimize(compute_cost_opt,[20,20,20,10], method='Nelder-Mead', args=(cost_data,context,diesel_co2), bounds=bornes)


1000


C:\Users\augus\AppData\Local\Temp\ipykernel_22568\3013852297.py:7: OptimizeWarning: Initial guess is not within the specified bounds
  resultat_optimisation = minimize(compute_cost_opt,[20,20,20,10], method='Nelder-Mead', args=(cost_data,context,diesel_co2), bounds=bornes)


1000


C:\Users\augus\AppData\Local\Temp\ipykernel_22568\3013852297.py:7: OptimizeWarning: Initial guess is not within the specified bounds
  resultat_optimisation = minimize(compute_cost_opt,[20,20,20,10], method='Nelder-Mead', args=(cost_data,context,diesel_co2), bounds=bornes)


KeyboardInterrupt: 

In [69]:
print(co2_cost)

45


In [98]:
0.75*24


18.0

In [81]:
df = pd.DataFrame(data=cost_data, index=[0])
df = (df.T)
df
# print (df)

# df.to_excel('dict1.xlsx')

,0
unserved_energy_cost,3000.00
diesel_variable_cost,177.00
solar_cost,747000.00
wind_cost,1300000.00
diesel_cost,1050505.00
storage_power_cost,0.00
storage_energy_cost,1167144.00
solar_om_cost,11000.00
wind_om_cost,40000.00
diesel_om_cost,101000.00


# Export en Excel 

In [96]:
with pd.ExcelWriter('sensibilite_CO2_cost.xlsx') as writer :
    cost_data_frame=  pd.DataFrame(data=cost_data, index=[0])
    cost_data_frame.to_excel(writer, sheet_name="000 Cost Data")

    prod_0 = [24.2061336 , 41.46341473, 20.42981495 , 0.25236392]
    prod_1 = [26.21368903 ,46.53202333 ,20.06925953 , 1.39625918]
    prod_2 = [28.08317237 ,52.00772941 ,19.69940091 , 2.33913029]
    prod_3 = [37.51803763 ,80.80808074 ,17.38383848 ,14.40692666]
    lCO2_cost = [0,45,100,500,1000]
    prod_4 = [ 47.64766584 ,100.       ,   16.      ,    27.51741351]
    L = [prod_0,prod_1,prod_2,prod_3,prod_4]

    for i in range(5) : 
        prod_opt = L[i]
        capacity_data = {'solar_gen':prod_opt[0],'wind_gen':prod_opt[1],'diesel_gen':prod_opt[2],'Energy':prod_opt[3],'diesel_param':diesel_param,'stock0':stock0}
        [solar_gen,wind_gen,diesel_gen,Energy]=prod_opt
        # Calcul des couts du scenario optimisé
        cost_opt = compute_cost(context,capacity_data,cost_data) 
        # Calcul du scenario énergétique 
        energy_scenario_opt =compute_data_opt(context,solar_gen,wind_gen,diesel_gen,Energy)
        capacity_data_frame = (pd.DataFrame(data=capacity_data, index=[0])).T
        cost_opt_frame = (pd.DataFrame(data=cost_opt, index=[0])).T
        capacity_data_frame.to_excel(writer ,sheet_name="{} Capacity Data".format(lCO2_cost[i]))
        cost_opt_frame.to_excel(writer ,sheet_name="{} Optimised Cost".format(lCO2_cost[i]))

